# Scrap notebook for testing and debugging

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [1]:
from data_loader import Data_Loader
import time
from scraper import Reddit

In [ ]:
# A list of urls that of submissions that you want to add to your graph. 
# These should be top level posts (not links to comments)
submissions = ['https://www.reddit.com/r/sanfrancisco/comments/7r3cy3/how_the_san_francisco_school_lottery_works_and/' ]
# Below is the full list of submissions I'm currently using for the school choice project
# submissions = [
#     'https://www.reddit.com/r/sanfrancisco/comments/bs5f69/just_had_the_elementary_school_lottery_explained/',
#     'https://www.reddit.com/r/sanfrancisco/comments/7r3cy3/how_the_san_francisco_school_lottery_works_and/',
#     'https://www.reddit.com/r/sanfrancisco/comments/4ah4no/fuck_the_sf_school_lottery_thats_all/',
#     'https://www.reddit.com/r/sanfrancisco/comments/b5kbse/how_the_student_assignment_system_works_sfusd/',
#     'https://www.reddit.com/r/sanfrancisco/comments/9hh9z8/two_sf_school_board_members_to_introduce/',
#     'https://www.reddit.com/r/sanfrancisco/comments/4646v8/experience_with_enrolling_in_sfusd_school/',
#     'https://www.reddit.com/r/sanfrancisco/comments/a5nrej/sf_school_board_plans_to_replace_muchcriticized/',
#     'https://www.reddit.com/r/sanfrancisco/comments/bhcxhb/san_francisco_had_an_ambitious_plan_to_tackle/',
#     'https://www.reddit.com/r/sanfrancisco/comments/5e5834/i_made_a_website_of_sf_elementary_school_test/',
#     'https://www.reddit.com/r/sanfrancisco/comments/cg5coh/sfusd_kindergarten/'
# ]

In [2]:
dl = Data_Loader()

In [3]:
dl.clear_graph()

In [4]:
comment_url = ['https://www.reddit.com/r/sanfrancisco/comments/7r3cy3/how_the_san_francisco_school_lottery_works_and/dstzrg6?utm_source=share&utm_medium=web2x']

In [5]:
dl.load_from_comment(comment_url)

adding author: bloobityblurp
adding author: mrmagcore
adding author: Kache
adding author: scopa0304
adding author: seancarter90
adding author: dockingnow
adding author: Ohyoudidntknowftt
adding author: ispeakdatruf
adding author: redditeyedoc
adding author: yaaaaayPancakes
adding author: SFCitizenDotCom
adding author: nlcund
adding author: justageek
adding author: NotheBrain
adding author: ultralame
adding author: hokeyphenokey
adding author: ajw204
adding author: santacruisin
adding author: LiverpoolLOLs
adding author: cardifan


In [7]:
dl.load_by_user_and_subreddit(user_name = "justasapling", subreddit_name = "sanfrancisco", limit = 10)

adding author: justasapling
adding author: familynight
adding author: singlewall
adding author: Gangreless
adding author: ShinobiZilla
adding author: KhwarazmianShah
adding author: the3natural
adding author: schpyda
adding author: Commodore_Vanderbilt
adding author: boyworldwide
adding author: xDeranx
adding author: mm825
adding author: DoctorTrash
adding author: mhda037
adding author: RobertPower415
adding author: AdvancedAlgebra
adding author: WoolziesInTheDryer
adding author: poop_dawg
adding author: BeanDemon
adding author: LiraNuna
adding author: StretchFrenchTerry


In [6]:
import pprint

# assume you have a Reddit instance bound to variable `reddit`
submission = Reddit().get_submission(id='39zje0')
print(submission.title) # to make it non-lazy
pprint.pprint(dict(vars(submission)).get("bloopidy", -1))

reddit will soon only be available over HTTPS
-1


# Timing experiments

In [ ]:
dl.clear_graph()

In [ ]:
tic = time.perf_counter()
dl.load_submissions(submissions)
toc = time.perf_counter()
print(f"Added submission in {toc - tic:0.4f} seconds")

In [ ]:
dl.clear_graph()

In [ ]:
dl.graph.schema.drop_uniqueness_constraint("User", "id")
dl.graph.schema.drop_uniqueness_constraint("Submission", "id")
dl.graph.schema.drop_uniqueness_constraint("Subreddit", "id")

In [ ]:
tic = time.perf_counter()
dl.load_submissions(submissions)
toc = time.perf_counter()
print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")